In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
#import plotly.plotly as py
#py.set_credentials_file(username='raoshaheryarkhan', api_key='KswnKD2cSzUMp6zxf25p')
#import plotly.figure_factory as ff
import math
from datetime import datetime
import sys
import os

In [2]:
project_path = "PycharmProjects/Amazon-Mining"
# make sure to use position 1
sys.path.insert(1, project_path)
os.chdir("/Users\RSK\PycharmProjects\Amazon-Mining")
from src.data.json_loader import JSONLoader
os.chdir("/Users\q")

###### Please Set Paths Here

In [3]:
file_path_reviewData = 'Documents\DataMining/reviews_Electronics_5.json.gz'
file_path_metaData = 'Documents\DataMining/meta_Electronics.json.gz'
file_path_overallratings = 'PycharmProjects\Amazon-Mining\dataset\Product_Ratings.json'
loader = JSONLoader()

In [4]:
metaData = loader.load_data(file_path_metaData)

In [5]:
metaData.loc[:,'asin'].unique().shape[0] 

498196

In [6]:
metaData.isnull().sum()

asin                0
imUrl             175
description     38726
categories          0
title            7002
price          108503
salesRank      369490
related        131237
brand          355664
dtype: int64

Conclusion: 25.8% products have sales Rank

In [7]:
metadf = metaData[metaData['salesRank'].notnull()]

In [8]:
metadf = metadf[metadf.salesRank != {}].reset_index()

In [9]:
#metadf.loc[metadf['asin'] == 'B0097WOKY4'].values


In [10]:
salesData = metadf[['asin','title','description','categories','salesRank','price']].copy()

In [11]:
def get_cat(l):
    [(k, v)] = l.items()
    return k
def get_rank(l):
    [(k, v)] = l.items()
    return v
def replace_amp(l):
    return l.replace("&amp;", "&")
    

In [12]:
salesData['salesCat'] = salesData.salesRank.apply(get_cat)
salesData['rank'] = salesData.salesRank.apply(get_rank)


In [13]:
del salesData['salesRank']

In [14]:
salesData['salesCat'] = salesData.salesCat.apply(replace_amp)

In [15]:
#salesData.iloc[:60]

In [16]:
#salesData.loc[salesData['rank'] == 3003].values

In [17]:
testSales = salesData.groupby(['salesCat', 'rank']).size()
testSales = testSales.reset_index()

In [18]:
testSales = testSales.rename(columns={testSales.columns[2]: 'count'})

In [19]:
testSales.loc[testSales['count'] > 8]

,salesCat,rank,count
18594,Cell Phones & Accessories,89114,9
22227,Clothing,2921,9
22263,Clothing,19977,9
94251,Sports & Outdoors,2758,9
94252,Sports & Outdoors,3003,14
94628,Sports & Outdoors,60994,17


In [20]:
del salesData['categories']
del salesData['description']
del salesData['title']

##### Overall Rating Data

In [21]:
ratingData = pd.read_json(file_path_overallratings, orient='columns')

In [22]:
salesData2 = pd.merge(salesData, ratingData, on = "asin")

In [23]:
reviewData = loader.load_data(file_path_reviewData)

##### Review Count Data

In [24]:
reviewCountData = reviewData[['asin','reviewerID']].groupby(['asin'], as_index=False).count()
reviewCountData = reviewCountData.rename(columns = {'reviewerID':'reviewCount'})

In [25]:
salesData2 = pd.merge(salesData2, reviewCountData, on = "asin")

##### Helpful Votes Count

In [26]:
reviewData['helpfulVotes']= reviewData['helpful'].str[0]

In [27]:
reviewHelpfulData = reviewData[['asin','helpfulVotes']].groupby(['asin'], as_index=False).sum()


In [28]:
salesRankdf = pd.merge(salesData2, reviewHelpfulData, on = "asin")

In [29]:
salesRankdf.to_json('PycharmProjects\Amazon-Mining\dataset\SalesRank_Reviews.json')

#### Brand Popularity affect on Sales Rank

In [30]:
reviewers = reviewData[['asin','reviewerID']].copy()

In [31]:
brandDetails = metaData[['asin','brand']].copy()

In [32]:
brandDetails = pd.merge(reviewers, brandDetails, on = "asin")

In [33]:
brandDetails = brandDetails.dropna(axis=0, how='any')

In [34]:
NonNaNBrandDetails = brandDetails[['brand','reviewerID']].groupby(['brand'], as_index=False).count()
NonNaNBrandDetails = NonNaNBrandDetails.rename(columns = {'reviewerID':'popularity'})

In [35]:
brandDetails2 = brandDetails[['asin','brand','reviewerID']].groupby(['asin','brand'], as_index=False).count()

In [36]:
del brandDetails2['reviewerID']

In [37]:
brandPop = pd.merge(brandDetails2, NonNaNBrandDetails, on = "brand")

In [38]:
salesRankBranddf = pd.merge(salesData, brandPop, on = "asin")

In [39]:
salesRankBranddf.to_json('PycharmProjects\Amazon-Mining\dataset\SalesRank_Popularity.json')